In [1]:
import pandas as pd
import json
import dateutil.parser
import numpy as np

In [2]:
data = pd.read_csv("data_science_exchanges_returns.csv",sep='\t') 
#Filtered by reason from the ecxhanges data
returns = data.dropna(subset=['Reason Name'])
returns[returns['Reason Name'].str.contains('_size_does_not_fit_')]
# returns.shape - (91242, 8)
returns['Order nr'] = returns['Order nr'].astype(np.int32)

c:\users\akshaya balamurugan\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\akshaya balamurugan\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [64]:
#Filtered conversions for Feb 1st to Feb 28th out of 62482 records from data_science_test_conversions.json
conv_data = [json.loads(line) for line in open('transactions.json', encoding="utf8")]
len(conv_data)

4705

In [65]:
#Filtered events for Feb 1st to Feb 28th with eventType = 'size' (recommedation was made) out of 222791 records from 'data_science_test_events' with eventType = 'size'
events_data = [json.loads(line) for line in open('events.json', encoding="utf8")]
len(events_data)

20894

In [136]:
#Extract SKU IDs, UIDs and recommended size from events JSON
# events_dict={}
events_l = []

for i in range(len(events_data)):
    temp=events_data[i]
    try:
        uid = temp['_source']['uid']
        skuid= temp['_source']['response']['skuId']
        fys = temp['_source']['response']['fys']
        for c in range(len(fys)):
            if fys[c]['recommended']== True:
                rec_size = fys[c]['size']
                break
            else:
                rec_size = "NA"
    except:
        print(i)
    events_dict = {'uid':uid,'skuid':skuid,'rec_size':rec_size}
    events_l.append(events_dict)

print(len(events_l))

20894


In [309]:
events_frame = pd.DataFrame(events_l)
events_frame.head(10)
len(events_frame) #20894 events with recommeded size for viewed SKU ID by UID
# events_frame

20894

In [347]:
# Extract SKU IDs, UIDs and purchased size from conversions JSON
convs_l=[]

for i_ in range(len(conv_data)):
    skuid_=[]
    size_=[]
    trans_=[]
    temp_=conv_data[i_]
    try:
        uid_= temp_['_source']['uid']
        temp_1 = temp_['_source']['products']
        trans_= temp_['_source']['transaction']
        for i_1 in range(len(temp_1)):
            skuid_.append(temp_1[i_1]['sku'])
            size_.append(temp_1[i_1]['size'])
    except:
        print(i_)
        skuid_.append("NA")    
        size_.append("NA")
        trans_.append("NA")
#     str_size = ",".join(str(x) for x in size_)
#     str_skuid = ",".join(str(x) for x in skuid_)
    convs_dict={'uid':uid_,'skuid':skuid_,'pur_size':size_,'transaction':trans_}
#     convs_dict={'uid':uid_,'skuid':str_skuid,'rec_size':str_size,'transaction':trans_}
    convs_l.append(convs_dict)
    
print(len(convs_l))
convs_l[45]

4705


{'uid': 'unknown',
 'skuid': ['6B50EAA5D1DAB4GS', '79E9EAA7B2B538GS'],
 'pur_size': ['International XL', 'International XL'],
 'transaction': '221931962'}

In [263]:
convs_frame = pd.DataFrame(convs_l)
convs_frame.head(15)

a = convs_frame.drop(convs_frame.columns[0], axis=1)
# a.head(10)
b = convs_frame.drop(convs_frame.columns[1], axis=1)
# b.head(10)

In [266]:
skuid_r = a.skuid.apply(pd.Series).merge(a, left_index = True, right_index = True).drop(["skuid"], axis = 1).melt(id_vars = ['transaction','uid'], value_name = "skuid").drop("variable", axis = 1).dropna()
size_r = b.pur_size.apply(pd.Series).merge(b, left_index = True, right_index = True).drop(["pur_size"], axis = 1).melt(id_vars = ['transaction','uid'], value_name = "pur_size").drop("variable", axis = 1).dropna()

In [353]:
#8600 entries with each SKU ID, UID, transaction and Purchased Size
conv_proc_frame = pd.concat([skuid_r,size_r],axis=1)
conv_proc_frame = conv_proc_frame.loc[:,~conv_proc_frame.columns.duplicated()]
conv_proc_frame.head(5)
len(conv_proc_frame)

8600

In [394]:
joined_frame =  pd.merge(events_frame, conv_proc_frame, how = 'inner', on=['uid','skuid'])
joined_frame = joined_frame.drop_duplicates(keep='first', inplace=False)
len(joined_frame)
joined_frame.head(5)

,rec_size,skuid,uid,transaction,pur_size
0,UK 12,3DAA0AABEBC5BDGS,f20f9aff-650d-49d9-9e6a-f692408dd254,282242962,UK 8
3,NA,BAB9CAA0751876GS,0c349735-c293-4731-ae90-451b8a1da885,221356962,International M
10,INT S,DC36EAA41E4DE5GS,84f75922-300e-487a-b5bc-401ed94c2648,287446962,International S
24,INT XL,9E755AA95CCA32GS,2bee3c8f-ae5a-40c2-a3a7-39a66f26bbab,288373962,International L
26,NA,8262BAAD3340B1GS,047b4f95-6e0a-47f9-9d21-a731a930653c,221633962,UK 12


In [396]:
joined_frame["pur_size"]= joined_frame["pur_size"].str.replace('International', 'INT')  
joined_frame["pur_size"]= joined_frame["pur_size"].str.replace('AUS', 'AU')  
# joined_frame

In [397]:
joined_frame['status'] = (joined_frame['pur_size'] == joined_frame['rec_size'])
# joined_frame

In [398]:
joined_frame["status"].value_counts()

False    76
True     52
Name: status, dtype: int64

In [399]:
#Total SKU IDs from returns
ret_sku_ids=returns['SKU ID'].tolist()
print(len(ret_sku_ids))
joined_frame.head(5)

91242


,rec_size,skuid,uid,transaction,pur_size,status
0,UK 12,3DAA0AABEBC5BDGS,f20f9aff-650d-49d9-9e6a-f692408dd254,282242962,UK 8,False
3,NA,BAB9CAA0751876GS,0c349735-c293-4731-ae90-451b8a1da885,221356962,INT M,False
10,INT S,DC36EAA41E4DE5GS,84f75922-300e-487a-b5bc-401ed94c2648,287446962,INT S,True
24,INT XL,9E755AA95CCA32GS,2bee3c8f-ae5a-40c2-a3a7-39a66f26bbab,288373962,INT L,False
26,NA,8262BAAD3340B1GS,047b4f95-6e0a-47f9-9d21-a731a930653c,221633962,UK 12,False


In [400]:
#SKU IDs from conversions for which size was recommended 
p_sku_ids = joined_frame['skuid'].tolist()
print(len(p_sku_ids))

128


In [401]:
#Returned SKU IDs and purchased SKU IDs
matches = set(p_sku_ids) & set(ret_sku_ids)
p_returned = list(matches)
len(p_returned)

98

In [410]:
true_sku = []
false_sku= []

for index, row in joined_frame.iterrows():
#     print(row)
    if row['status'] == True:
        true_sku.append(row['skuid'])
    else:
        false_sku.append(row['skuid'])

tot_true = len(true_sku)
tot_false = len(false_sku)

In [416]:
m = set(true_sku) & set(p_returned)
mat_true = (len(list(m)))

In [417]:
m1 = set(false_sku) & set(p_returned)
mat_false = len(list(m1))

In [420]:
#Choice same as recommendation - Sales/Return Rate
ret_rate_rec_true=mat_true/tot_true
ret_rate_rec_true

0.8461538461538461

In [421]:
#Choice not same as recommendation - - Sales/Return Rate
ret_rate_rec_false=mat_false/tot_false
ret_rate_rec_false

0.8026315789473685